In [1]:
#hide
#skip
%config Completer.use_jedi = False
%config IPCompleter.greedy=True
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display=Display(visible=0,size=(400,300))
    display.start()

In [3]:
# Python native modules
import os,sys
from copy import deepcopy,copy
from typing import *
import types
import logging
import inspect
from itertools import chain
from functools import partial
# Third party libs
from fastcore.all import *
import numpy as np
# Local modules
from fastrl.fastai.loop import *

_logger=logging.getLogger(__name__)

In [4]:
class Outer(Loop):
    @event
    def before_step(self) :  print('before_step')
    @event
    def on_step(self)     :  print('on_step')
    @event
    def after_step(self)  :  print('after_step')
    @event
    def failed_step(self) :  print('failed_step')
    @event
    def finally_step(self):  print('finally_step')
 
    @event
    def before_jump(self) :  print('before_jump')
    @event
    def on_jump(self)     :  print('on_jump')
    @event
    def after_jump(self)  :  print('after_jump')
    @event
    def failed_jump(self) :  print('failed_jump')
    @event
    def finally_jump(self):  print('finally_jump')

class Inner(Loop):
    call_on=L(Outer.on_step,Outer.after_step,Outer.finally_jump)
    
    @event
    def before_iteration(self) : print('before_iteration')
    @event
    def on_iteration(self)     : print('on_iteration')
    @event
    def after_iteration(self)  : print('after_iteration')
    @event
    def failed_iteration(self) : print('failed_iteration')
    @event
    def finally_iteration(self): print('finally_iteration')
    # def run(self,loops=None,cbs=None):
    #     sections=self.get_sections(loops=loops,cbs=cbs)
    #     for section in sections:
    #         section.run()
    
class FailingInner(Loop):
    call_on=L(Inner.finally_iteration)
    
    @event
    def on_force_fail(self):                    
        print('on_force_fail')
        raise Exception
        

In [5]:
class OuterCallback(Callback):
    call_on=L(Outer.on_step)
    
    def before_iteration(self)->dict(this=list,that=str):
        print('   OuterCallback called lol')

Outer().get_sections(L(Inner(),FailingInner()),OuterCallback)

[Section, Section]

In [6]:
try:
    Outer().run(L(Inner(),FailingInner()),OuterCallback())
finally:
    exception_info=sys.exc_info()
    print(sys.exc_info())

before_step
on_step
before_iteration
on_iteration
after_iteration
finally_iteration
on_force_fail
adding render traceback
failed_step
finally_step
(<class 'Exception'>, Exception(), <traceback object at 0x7fa02df46b40>)
(<class 'Exception'>, Exception(), <traceback object at 0x7fa02df46b80>)
hi lol
{'self': <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7fa0677b63a0>, 'code_obj': <code object <module> at 0x7fa0644ad030, file "/tmp/ipykernel_707/1928053429.py", line 1>, 'result': <ExecutionResult object at 7fa02df8f6d0, execution_count=6 error_before_exec=None error_in_exec= info=<ExecutionInfo object at 7fa02df8f520, raw_cell="try:
    Outer().run(L(Inner(),FailingInner()),Out.." store_history=True silent=False shell_futures=True> result=None>, 'async_': False, '__tracebackhide__': '__ipython_bottom__', 'old_excepthook': <bound method IPKernelApp.excepthook of <ipykernel.kernelapp.IPKernelApp object at 0x7fa06cf32340>>, 'outflag': True}


Exception: 

In [ ]:

# Outer().get_sections(L(Inner(),FailingInner()),OuterCallback)[0].events['on_inner'][0].sections[0].run()

In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script('[!02_fastai.loop.old]*.ipynb')
    notebook2html('[!02_fastai.loop.old]*.ipynb')